# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff384dd9400>
├── 'a' --> tensor([[-1.3283, -1.0591, -1.3756],
│                   [ 0.3170,  2.2281, -0.8048]])
└── 'x' --> <FastTreeValue 0x7ff384dd9b50>
    └── 'c' --> tensor([[-0.4544, -1.1829, -1.1726,  1.7738],
                        [ 0.9765,  0.7680,  1.3578,  0.3508],
                        [-0.0104, -0.3855,  0.2148,  1.5169]])

In [4]:
t.a

tensor([[-1.3283, -1.0591, -1.3756],
        [ 0.3170,  2.2281, -0.8048]])

In [5]:
%timeit t.a

68.3 ns ± 0.994 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff384dd9400>
├── 'a' --> tensor([[ 0.6648, -1.5039, -0.0675],
│                   [-0.3613, -0.2289,  1.3788]])
└── 'x' --> <FastTreeValue 0x7ff384dd9b50>
    └── 'c' --> tensor([[-0.4544, -1.1829, -1.1726,  1.7738],
                        [ 0.9765,  0.7680,  1.3578,  0.3508],
                        [-0.0104, -0.3855,  0.2148,  1.5169]])

In [7]:
%timeit t.a = new_value

83.5 ns ± 1.08 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.3283, -1.0591, -1.3756],
               [ 0.3170,  2.2281, -0.8048]]),
    x: Batch(
           c: tensor([[-0.4544, -1.1829, -1.1726,  1.7738],
                      [ 0.9765,  0.7680,  1.3578,  0.3508],
                      [-0.0104, -0.3855,  0.2148,  1.5169]]),
       ),
)

In [10]:
b.a

tensor([[-1.3283, -1.0591, -1.3756],
        [ 0.3170,  2.2281, -0.8048]])

In [11]:
%timeit b.a

61.1 ns ± 1.37 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7161,  0.0428,  0.2370],
               [ 0.6036, -1.4366,  1.1933]]),
    x: Batch(
           c: tensor([[-0.4544, -1.1829, -1.1726,  1.7738],
                      [ 0.9765,  0.7680,  1.3578,  0.3508],
                      [-0.0104, -0.3855,  0.2148,  1.5169]]),
       ),
)

In [13]:
%timeit b.a = new_value

570 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

941 ns ± 20.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 300 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

261 µs ± 7.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

264 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff35c2c00a0>
├── 'a' --> tensor([[[-1.3283, -1.0591, -1.3756],
│                    [ 0.3170,  2.2281, -0.8048]],
│           
│                   [[-1.3283, -1.0591, -1.3756],
│                    [ 0.3170,  2.2281, -0.8048]],
│           
│                   [[-1.3283, -1.0591, -1.3756],
│                    [ 0.3170,  2.2281, -0.8048]],
│           
│                   [[-1.3283, -1.0591, -1.3756],
│                    [ 0.3170,  2.2281, -0.8048]],
│           
│                   [[-1.3283, -1.0591, -1.3756],
│                    [ 0.3170,  2.2281, -0.8048]],
│           
│                   [[-1.3283, -1.0591, -1.3756],
│                    [ 0.3170,  2.2281, -0.8048]],
│           
│                   [[-1.3283, -1.0591, -1.3756],
│                    [ 0.3170,  2.2281, -0.8048]],
│           
│                   [[-1.3283, -1.0591, -1.3756],
│                    [ 0.3170,  2.2281, -0.8048]]])
└── 'x' --> <FastTreeValue 0x7ff2d8ab8850>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

43.6 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff35c3a8160>
├── 'a' --> tensor([[-1.3283, -1.0591, -1.3756],
│                   [ 0.3170,  2.2281, -0.8048],
│                   [-1.3283, -1.0591, -1.3756],
│                   [ 0.3170,  2.2281, -0.8048],
│                   [-1.3283, -1.0591, -1.3756],
│                   [ 0.3170,  2.2281, -0.8048],
│                   [-1.3283, -1.0591, -1.3756],
│                   [ 0.3170,  2.2281, -0.8048],
│                   [-1.3283, -1.0591, -1.3756],
│                   [ 0.3170,  2.2281, -0.8048],
│                   [-1.3283, -1.0591, -1.3756],
│                   [ 0.3170,  2.2281, -0.8048],
│                   [-1.3283, -1.0591, -1.3756],
│                   [ 0.3170,  2.2281, -0.8048],
│                   [-1.3283, -1.0591, -1.3756],
│                   [ 0.3170,  2.2281, -0.8048]])
└── 'x' --> <FastTreeValue 0x7ff2d8ad4d30>
    └── 'c' --> tensor([[-0.4544, -1.1829, -1.1726,  1.7738],
                        [ 0.9765,  0.7680,  1.3578,  0.3508],
                 

In [23]:
%timeit t_cat(trees)

38.3 µs ± 862 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.2 µs ± 525 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.4544, -1.1829, -1.1726,  1.7738],
                       [ 0.9765,  0.7680,  1.3578,  0.3508],
                       [-0.0104, -0.3855,  0.2148,  1.5169]],
              
                      [[-0.4544, -1.1829, -1.1726,  1.7738],
                       [ 0.9765,  0.7680,  1.3578,  0.3508],
                       [-0.0104, -0.3855,  0.2148,  1.5169]],
              
                      [[-0.4544, -1.1829, -1.1726,  1.7738],
                       [ 0.9765,  0.7680,  1.3578,  0.3508],
                       [-0.0104, -0.3855,  0.2148,  1.5169]],
              
                      [[-0.4544, -1.1829, -1.1726,  1.7738],
                       [ 0.9765,  0.7680,  1.3578,  0.3508],
                       [-0.0104, -0.3855,  0.2148,  1.5169]],
              
                      [[-0.4544, -1.1829, -1.1726,  1.7738],
                       [ 0.9765,  0.7680,  1.3578,  0.3508],
                       [-0.0104, -0.3855,  0.2148,  1.5169]],

In [26]:
%timeit Batch.stack(batches)

96.2 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.4544, -1.1829, -1.1726,  1.7738],
                      [ 0.9765,  0.7680,  1.3578,  0.3508],
                      [-0.0104, -0.3855,  0.2148,  1.5169],
                      [-0.4544, -1.1829, -1.1726,  1.7738],
                      [ 0.9765,  0.7680,  1.3578,  0.3508],
                      [-0.0104, -0.3855,  0.2148,  1.5169],
                      [-0.4544, -1.1829, -1.1726,  1.7738],
                      [ 0.9765,  0.7680,  1.3578,  0.3508],
                      [-0.0104, -0.3855,  0.2148,  1.5169],
                      [-0.4544, -1.1829, -1.1726,  1.7738],
                      [ 0.9765,  0.7680,  1.3578,  0.3508],
                      [-0.0104, -0.3855,  0.2148,  1.5169],
                      [-0.4544, -1.1829, -1.1726,  1.7738],
                      [ 0.9765,  0.7680,  1.3578,  0.3508],
                      [-0.0104, -0.3855,  0.2148,  1.5169],
                      [-0.4544, -1.1829, -1.1726,  1.7738],
                   

In [28]:
%timeit Batch.cat(batches)

175 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

554 µs ± 26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
